In [68]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import sys
import random
import string
import re
from ipywidgets import IntProgress
from IPython.display import display

Функция прогресса итерации:

In [69]:
def log_progress(sequence, every=1):
    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record   

Feature toggles:

In [70]:
act_filter = True  # фильтр по актуальности выпуска (FT)
sim_only = True  # фильтр по sim (FT)

Гиперпараметры:

In [71]:
min_release_year = 2018  # год отсечки по актуальности
tac_length = 8  # маска по длине tac кода

Собираем файлы в один фрейм из каталога ЯМ с атрибутом производителя:

In [72]:
list_of_files = os.listdir('data/top_ym')
market_features = ['Производитель']
df_m = pd.DataFrame()

for item in log_progress(list_of_files, every=1):    
    path = 'data/top_ym/' + str(item)    
    df_file = pd.read_csv(path, delimiter=';', encoding='windows-1251', usecols=market_features)
    df_m = pd.concat([df_m, df_file], axis=0)     

IntProgress(value=0, max=313)

Переименовываем признак производителя:

In [73]:
df_m = df_m.rename(columns={'Производитель': 'ven_m', 'Категория': 'class_m'})

Убираем пробелы и приводим к нижнему регистру:

In [74]:
df_m = df_m.applymap(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)
df_m = df_m.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_m = df_m.applymap(lambda x: x.lower() if isinstance(x, str) else x)

Составляем сет производителей ЯМаркета:

In [75]:
market_vendors = list(set(df_m['ven_m']))

Читаем каталог девайсов с признаками: 'tac', 'model_name', 'vendor_name', 'sim_count', 'release_date':

In [76]:
dev_features = ['TAC', 'MODEL_NAME', 'VENDOR_NAME', 'SIM_COUNT', 'RELEASE_DATE']
df = pd.read_csv('data/devices_dirty.csv', usecols=dev_features) 

Число записей:

In [77]:
dim_before = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 165963


Переименовываем столбцы:

In [78]:
column_voc = {'TAC': 'tac_d', 'VENDOR_NAME': 'ven_d', 'MODEL_NAME': 'model_d', 'SIM_COUNT': 'sim_d', 'RELEASE_DATE': 'date_d'}
df = df.rename(columns=column_voc)

Меняем порядок столбцов и выводим структуру:

In [79]:
df = df[['ven_d', 'model_d', 'tac_d', 'sim_d', 'date_d']]
df.head(0)

,ven_d,model_d,tac_d,sim_d,date_d


Показываем пропуски в относительных долях:

In [80]:
print('Число нулей для всех признаков:', dict(round(df.isnull().sum()/len(df), 2)))

Число нулей для всех признаков: {'ven_d': 0.0, 'model_d': 0.0, 'tac_d': 0.0, 'sim_d': 0.65, 'date_d': 0.0}


Показываем пропуски в абсолютных значения:

In [81]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'ven_d': 0, 'model_d': 0, 'tac_d': 0, 'sim_d': 108350, 'date_d': 57}


Устройства в сим картой в %:

In [82]:
with_sim = df[df['sim_d'] > 0]['model_d'].count()/df['model_d'].count()
print('Устройства c сим картой - доля от всех моделей:', int(round(100*with_sim)), '%')

Устройства c сим картой - доля от всех моделей: 35 %


Пропусков по дате: < 1% от числа записей (ошибка). Сброс записей с пропущенными значениями в ключевых атрибутах (фильтр I):

In [83]:
df.dropna(subset=['ven_d', 'model_d', 'tac_d', 'date_d'], how='any', axis=0, inplace=True)

Преобразование строковых типов:

In [84]:
df = df.astype({'ven_d': 'str', 'model_d': 'str', 'tac_d': 'str'})

Преобразование целых типов и даты:

In [85]:
df['sim_d'] = df['sim_d'].fillna(-1).astype(int).replace(-1, np.nan)
df['date_d'] = pd.to_datetime(df['date_d']).dt.year.astype(int)

Выводим число нулей:

In [86]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'ven_d': 0, 'model_d': 0, 'tac_d': 0, 'sim_d': 108295, 'date_d': 0}


Удаляем знаки пунктуации в названии модели и производителе:

In [87]:
for item in list(string.punctuation): 
    df[['model_d', 'ven_d']] = df[['model_d', 'ven_d']].applymap(lambda x: x.replace(item, ' '))

Удаляем пробелы:

In [88]:
df = df.applymap(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x).applymap(lambda x: x.lower() if isinstance(x, str) else x)

Удаляем из названий моделей производителей:

In [89]:
for item in log_progress(list(set(df['ven_d'])), every=1):
    df['model_d'] = df['model_d'].apply(lambda x: x.replace(item, ''))

IntProgress(value=0, max=6839)

Оставляем записи с производителями, которые есть в каталоге ЯМ (фильтр II):

In [90]:
df = df[df['ven_d'].isin(market_vendors)]

Удаляем записи без названия модели после очистки:

In [91]:
df.dropna(subset=['model_d'], how='any', inplace=True)

Создаем уникальный признак 'ven_model_d':  

In [92]:
df['ven_model_d'] = df['ven_d'] + ' ' + df['model_d']
df = df[['ven_d', 'model_d', 'ven_model_d', 'tac_d', 'sim_d', 'date_d']]

Оставляем девайсы с sim картой  (фильтр III):

In [93]:
if sim_only is True: 
    df = df[df['sim_d'] >= 1]

Создаем фрейм с датой выпуска и уникальной моделью в атрибутах:

In [94]:
df_actual = df[['ven_model_d', 'date_d']]
df_actual = df_actual.drop_duplicates()

Определяем актуальность каталога:

In [95]:
print('Мода по дате выпуска:', int(df_actual['date_d'].mode()))
print('Минимальная дата выпуска:', int(df_actual['date_d'].min()))
print('Максимальная дата выпуска:', int(df_actual['date_d'].max()))

Мода по дате выпуска: 2015
Минимальная дата выпуска: 1992
Максимальная дата выпуска: 2019


<img align='left' src='images/year_cpr.png' width='400'>

Отфильтровываем записи с годом выпуска раньше чем min_release_year  (фильтр IV):

In [96]:
if act_filter is True: 
    df = df[df['date_d'] >= min_release_year]

Удаляем атрибут с датой выпуска:

In [97]:
df = df.drop(columns=['date_d'])

Выводим структуру:

In [98]:
df.head(0)

,ven_d,model_d,ven_model_d,tac_d,sim_d


Оставляем записи с корректной длиной tac (фильтр V):

In [99]:
df = df[df['tac_d'].apply(len) == tac_length]

Сбрасываем записи с нулями:

In [100]:
df.dropna(axis=0, how='any', inplace=True)
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'ven_d': 0, 'model_d': 0, 'ven_model_d': 0, 'tac_d': 0, 'sim_d': 0}


Добавляем в датафрейм признак с числом tac номеров на модель:

In [101]:
df_tac_num = df[['ven_model_d', 'tac_d']].groupby('ven_model_d', as_index=False).count().rename(columns={'tac_d': 'tac_num_d'})
df = pd.merge(df, df_tac_num, on=['ven_model_d'], how='inner')

n(tacs)<=n(sims). Исключаем записи, которые нарушают данное правило (фильтр VI): 

In [102]:
df = df[df['tac_num_d'] <= df['sim_d']]

Неоднозначность {tac => model}:

In [103]:
df_tac = df[['ven_model_d', 'tac_d']].groupby('tac_d').count()
print('На один tac приходится max', df_tac['ven_model_d'].max(), 'устройств(о)')

На один tac приходится max 1 устройств(о)


Неоднозначность {model => tac}:

In [104]:
df_mod = df[['ven_model_d', 'tac_d']].groupby('ven_model_d').count()
print('На одно устройство приходится max', round(df_mod['tac_d'].max()), 'tac(s)')

На одно устройство приходится max 2 tac(s)


Неоднозначность {vendor => model}:

In [105]:
df_tac = df[['ven_d', 'ven_model_d']].groupby('ven_d').count()
print('На одного производителя приходится max', df_tac['ven_model_d'].max(), 'моделей')

На одного производителя приходится max 214 моделей


Сбрасываем вспомогательные признаки:

In [106]:
df = df.drop(columns=['sim_d', 'tac_num_d', 'ven_model_d'])

Показываем структуру:

In [107]:
df.head(0)

,ven_d,model_d,tac_d


Статистики по распределению числа слов в наименовании модели:

In [108]:
print('Среднее число слов:', round(df['model_d'].str.split().apply(len).mean()))
print('Разброс числа слов:', round(df['model_d'].str.split().apply(len).max()-df['model_d'].str.split().apply(len).min()))

Среднее число слов: 4
Разброс числа слов: 11


Статистики по распределению числа символов в наименовании модели:

In [109]:
print('Среднее число символов:', round(df['model_d'].apply(len).mean()))
print('Разброс числа символов:', round(df['model_d'].apply(len).max()-df['model_d'].apply(len).min()))

Среднее число символов: 18
Разброс числа символов: 57


Структура:

In [110]:
df.head(0)

,ven_d,model_d,tac_d


Число записей:

In [111]:
dim_after = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 684


Компрессия по числу записей:

In [112]:
print('Компрессия по числу записей:', round(dim_before/dim_after))

Компрессия по числу записей: 243


Описание фрейма:

In [113]:
df.describe()[:2][:]

,ven_d,model_d,tac_d
count,684,684,684
unique,27,566,684


Записываем датафрейм в туже папку:

In [114]:
df.to_csv('data/devices_clean.csv', index=False)